##### Por hacer:

Crear más instancias

Analizar qué datos serán necesarios guardar y correr las instancias

Reevaluar la función de cierre de ciclos

##### Notas

Se debería añadir utilidad al llevar materias básicas sobre las de elección libre?

-Al darle un peso mayor a la utilidad de cerrar ciclos, ¿mejorarán los resultados?

-¿Por qué el mejor resultado teórico es mejor que el resultado práctico a pesar de tener mejor desempeno?

El desempeño de la poblacion siempre se estanca porque terminan habiendo muchas cargas inválidas

La utilidad de cerrar ciclos no tiene suficiente relevancia sobre las otras funciones debido a la naturaleza de su cálculo.

##### Cambios:

-Se modificó "obtenerOfertaUtil" para excluir materias de elección libre ya aprobadas de primeros 4 semestres.

-Se cambio función de utilidad disponibilidad de horario

-Error en "RespetaSeriacion", si la materia necesaria ya había sido aprobada retornaba false

In [1]:
import pandas as pd
import numpy as np
import pygad as pg
import time
import requests
import matplotlib.pyplot as plt

# Limpieza y preparación de los datos

## Lectura de datos

In [2]:
d16 = pd.read_csv('./datos/datos_2016.csv')
d17 = pd.read_csv('./datos/datos_2017.csv')
d18 = pd.read_csv('./datos/datos_2018.csv')
d19 = pd.read_csv('./datos/datos_2019.csv')
d2021 = pd.read_csv('./datos/2020_2021.csv')

In [3]:
asignaturas = pd.read_csv('asignaturas2.csv',encoding = 'ISO-8859-1')

In [4]:
plan_2016 = asignaturas.query('plan == "IDP2016"')

In [5]:
oferta = pd.read_csv('oferta_ideio.csv',encoding = 'ISO-8859-1')

In [6]:
seriaciones = pd.read_csv('seriacion.csv')

## Visualización de datos

In [7]:
d16.head()

,Unnamed: 0,carrera,periodo,clave,asignatura,promediofinal,docente,ceveval_global,ceneval_analitico,ceneval_matematico,...,r_modulo8,modulo9,r_modulo9,modulo10,r_modulo10,modulo11,r_modulo11,modulo12,r_modulo12,n_matricula
0,0,Negocios Internacionales,201601,DP0295,Taller de formación en responsabilidad social ...,10.0,Maldonado Saldaña / Gisela,1150.0,1108.0,1228.0,...,-,Cálculo,-,Inglés,-,Administración,-,Informática y Computación,-,140311285
1,1,Negocios Internacionales,201601,NI0215,Taller de software para negocios internacionales,9.0,Villeda Cuellar / Víctor Hugo,1150.0,1108.0,1228.0,...,-,Cálculo,-,Inglés,-,Administración,-,Informática y Computación,-,140311285
2,2,Negocios Internacionales,201601,LI1104,Nivel 4 Inglés,10.0,Ross / Andrew Patrick Simon,1150.0,1108.0,1228.0,...,-,Cálculo,-,Inglés,-,Administración,-,Informática y Computación,-,140311285
3,3,Negocios Internacionales,201601,NI0209,Administración financiera,6.0,Vallejo Filoteo / Jorge,1150.0,1108.0,1228.0,...,-,Cálculo,-,Inglés,-,Administración,-,Informática y Computación,-,140311285
4,4,Negocios Internacionales,201601,NI0213,Transporte y logística en comercio exterior,9.0,Cañedo Magaña / Magdalena del Carmen,1150.0,1108.0,1228.0,...,-,Cálculo,-,Inglés,-,Administración,-,Informática y Computación,-,140311285


In [8]:
d2021.head()

,Unnamed: 0,Column1,programa,periodo,clave,asignatura,docente,promediofinal,ceveval_global,hrs_trabaja,n_matricula
0,0,1,Ingeniería en Datos e Inteligencia Organizacional,202001,IL0414,Taller de argumentación y técnicas de negociación,Torres Pérez / Rubén Juan,7.0,952.0,De 5 a 10 horas,170311163
1,1,2,Ingeniería en Datos e Inteligencia Organizacional,202001,II0422,Taller de emprendedores,Zapata Solís / Zuemy De Lourdes,8.0,1048.0,No trabajaba,180311454
2,2,3,Ingeniería en Datos e Inteligencia Organizacional,202001,DP0498,Problemas de México,Díaz Molina / Libertad Fidelina,9.0,1144.0,De 5 a 10 horas,170311232
3,3,4,Ingeniería en Datos e Inteligencia Organizacional,202001,IT0264,Introducción a los sistemas de información,Olivares González / Oscar,10.0,1072.0,De 11 a 15 horas,190311813
4,4,5,Ingeniería en Datos e Inteligencia Organizacional,202001,II0215,Electricidad y magnetismo,Ramírez Carmona / Leslye Johanna,7.0,1018.0,Más de 20 horas,180311416


In [9]:
asignaturas.head()

,ciclos,clave,nombre,creditos,tipo,plan,preespecialidad
0,1,II0106,Cálculo diferencial,8,Basica,IAP2019,NaN
1,1,II0209,Cálculo integral,6,Basica,IAP2019,NaN
2,1,DP0194,Desarrollo de habilidades del pensamiento,6,Basica,IAP2019,NaN
3,1,IA0106,Ecología,6,Basica,IAP2019,NaN
4,1,II0102,Física experimental,8,Basica,IAP2019,NaN


In [10]:
plan_2016.head()

,ciclos,clave,nombre,creditos,tipo,plan,preespecialidad
603,1,IT0101,Algoritmos y estructura de datos,6,Basica,IDP2016,NaN
604,1,II0106,Cálculo diferencial,8,Basica,IDP2016,NaN
605,1,II0209,Cálculo integral,6,Basica,IDP2016,NaN
606,1,DP0194,Desarrollo de habilidades del pensamiento,6,Basica,IDP2016,NaN
607,1,ID0101,Diseño de patrones para datos estructurados,6,Basica,IDP2016,NaN


In [11]:
oferta.head()

,Ciclo,Trayectoria,Tipo,Clave,Nombre,Maestro,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
0,1,0,BAS,DP0193,Taller de redacción profesional,Alvarado Martínez / María Del Carmen,-,-,17:00-18:00,-,16:00-18:00,-
1,1,0,BAS,DP0191,Taller de interacción grupal y competencias co...,Diaz Barriga Cuevas / Claudia,-,-,-,15:00-17:00,17:00-18:00,-
2,1,0,BAS,DP0191,Taller de interacción grupal y competencias co...,Diaz Barriga Cuevas / Claudia,18:00-20:00,-,-,17:00-18:00,-,-
3,1,0,BAS,DP0193,Taller de redacción profesional,Parra Dorantes / Roberto,-,-,18:00-20:00,-,17:00-18:00,-
4,1,0,BAS,DP0194,Desarrollo de habilidades del pensamiento,Mendoza Luna / José Arturo,-,-,19:00-20:00,-,20:00-22:00,-


In [12]:
seriaciones.head()

,plan,ser1,ser2
0,IDP2016,II0106,II0209
1,IDP2016,ID0102,II0215
2,IDP2016,II0209,II0320
3,IDP2016,ID0103,IT0263
4,IDP2016,IT0264,ID0204


## Limpieza de datos

In [13]:
cargas16_19 = pd.concat([d16,d17,d18,d19])
cargas20_21 = d2021

In [14]:
cargas16_19 = cargas16_19.query('carrera == "Ingeniería en Datos e Inteligencia Organizacional"')

In [15]:
cargas16_19 = cargas16_19[['Unnamed: 0', 'carrera', 'periodo', 'clave', 'asignatura',
       'promediofinal', 'docente', 'ceveval_global', 'ceneval_analitico',
       'ceneval_matematico', 'ceneval_lengua', 'ceneval_esp',
       'n_matricula']]

In [16]:
matriculas16 = cargas16_19.query('n_matricula > 160000000 and n_matricula < 170000000')['n_matricula'].unique()
matriculas17 = cargas16_19.query('n_matricula > 170000000 and n_matricula < 180000000')['n_matricula'].unique()
matriculas18 = cargas16_19.query('n_matricula > 180000000 and n_matricula < 190000000')['n_matricula'].unique()
matriculas19 = cargas16_19.query('n_matricula > 190000000 and n_matricula < 200000000')['n_matricula'].unique()
matriculas20 = cargas20_21.query('n_matricula > 200000000 and n_matricula < 210000000')['n_matricula'].unique()
matriculas21 = cargas20_21.query('n_matricula > 210000000 and n_matricula < 220000000')['n_matricula'].unique()

In [17]:
matriculas = np.concatenate((matriculas16, matriculas17,matriculas18,matriculas19,matriculas20,matriculas21))

In [18]:
cargas20_21 = cargas20_21.rename(columns={'programa': 'carrera'})

In [19]:
formato_cargas20_21 = cargas20_21[['carrera', 'periodo', 'clave', 'asignatura','docente','n_matricula','promediofinal']]
formato_cargas16_19 = cargas16_19[['carrera', 'periodo', 'clave', 'asignatura','docente','n_matricula','promediofinal']]

formato_cargas = pd.concat([formato_cargas16_19,formato_cargas20_21])

In [20]:
oferta = oferta[['Clave','Nombre', 'Maestro', 'Lunes','Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado']]

In [21]:
oferta = oferta.drop_duplicates()

In [22]:
oferta['Lunes'].unique()

array(['-', '18:00-20:00', '17:00-19:00', '20:00-22:00', '15:00-17:00',
       '9:00-11:00', '19:00-21:00', '19:00-20:00', '7:00-9:00',
       '11:00-13:00', '13:00-14:00', '16:00-18:00', '14:00-16:00',
       '7:00-10:00', '7:00-8:00', '16:00-19:00', '21:00-22:00',
       '16:00-17:00'], dtype=object)

In [23]:
oferta['Martes'].unique()

array(['-', '15:00-17:00', '20:00-21:00', '20:00-22:00', '19:00-20:00',
       '16:00-18:00', '18:00-19:00', '7:00-9:00', '9:00-11:00',
       '18:00-20:00', '14:00-16:00', '13:00-15:00', '8:00-9:00'],
      dtype=object)

In [24]:
oferta['Miercoles'].unique()

array(['17:00-18:00', '-', '18:00-20:00', '19:00-20:00', '16:00-17:00',
       '15:00-17:00', '20:00-21:00', '20:00-22:00', '18:00-19:00',
       '12:00-14:00', '7:00-9:00', '9:00-11:00', '11:00-12:00',
       '11:00-13:00', '\r\n13:00-15:00', '13:00-15:00', '17:00-19:00',
       '19:00-21:00', '15:00-16:00', '12:00-13:00', '8:00-10:00',
       '7:00-8:00', '10:00-12:00', '16:00-18:00', '14:00-15:00'],
      dtype=object)

In [25]:
oferta['Jueves'].unique()

array(['-', '15:00-17:00', '17:00-18:00', '20:00-21:00', '16:00-17:00',
       '9:00-10:00', '20:00-22:00', '18:00-20:00', '19:00-21:00',
       '7:00-8:00', '13:00-14:00', '8:00-9:00', '9:00-11:00',
       '16:00-18:00', '14:00-15:00', '13:00-15:00', '7:00-9:00',
       '15:00-16:00', '18:00-19:00', '19:00-20:00', '21:00-22:00'],
      dtype=object)

In [26]:
oferta['Viernes'].unique()

array(['16:00-18:00', '17:00-18:00', '-', '20:00-22:00', '15:00-17:00',
       '14:00-15:00', '18:00-20:00', '20:00-21:00', '13:00-14:00',
       '9:00-10:00', '7:00-9:00', '10:00-11:00', '16:00-17:00',
       '14:00-16:00', '18:00-19:00', '8:00-10:00', '19:00-20:00',
       '19:00-21:00'], dtype=object)

In [27]:
oferta.loc[36,'Miercoles'] = '13:00-15:00'

In [28]:
oferta['Miercoles'].unique()

array(['17:00-18:00', '-', '18:00-20:00', '19:00-20:00', '16:00-17:00',
       '15:00-17:00', '20:00-21:00', '20:00-22:00', '18:00-19:00',
       '12:00-14:00', '7:00-9:00', '9:00-11:00', '11:00-12:00',
       '11:00-13:00', '13:00-15:00', '17:00-19:00', '19:00-21:00',
       '15:00-16:00', '12:00-13:00', '8:00-10:00', '7:00-8:00',
       '10:00-12:00', '16:00-18:00', '14:00-15:00'], dtype=object)

In [29]:
def formatoHora(hora):
    if len(hora) == 11:
        return hora
    elif len(hora) == 10:
        return '0' + hora
    elif len(hora) == 9:
        return '0' + hora[0:5] + '0' + hora[5:]
    elif len(hora) == 1:
        return '-'

In [30]:
oferta['Lunes'] = oferta['Lunes'].apply(lambda horas: formatoHora(horas))
oferta['Martes'] = oferta['Martes'].apply(lambda horas: formatoHora(horas))
oferta['Miercoles'] = oferta['Miercoles'].apply(lambda horas: formatoHora(horas))
oferta['Jueves'] = oferta['Jueves'].apply(lambda horas: formatoHora(horas))
oferta['Viernes'] = oferta['Viernes'].apply(lambda horas: formatoHora(horas))

# Definición de funciones útiles

## Función para obtener un Kardex aleatorio

In [31]:
def obtenerKardex(mat = -1):
    if(mat == -1):
        mat = np.random.choice(matriculas)
    return formato_cargas.query('n_matricula == ' + str(mat)).sort_values('periodo')

In [32]:
def obtenerCreditos(kardex):
    claves = kardex.query('promediofinal >= 7')['clave'].unique()
    totalCreditos = 0
    for clave in claves:
        if(clave[0:2] == 'LI' or clave[0:2] == 'TA' or clave[0:2] == 'AD'):
            continue
        totalCreditos += plan_2016.query('clave == "' + clave + '"')['creditos'].values[0]
    return totalCreditos

## Obtención de oferta útil

En este sección se crea la función "obtenerOfertaUtil" la cual filtra la oferta académica para eliminar las materias que no cumplen con la restricción de llevar materias aprobadas y la restricción de llevar materias que no respeten la seriación.

In [33]:
def materiaHaSidoAprobada(kardex,clave):
    if len(kardex.query('clave == "' + clave + '" and promediofinal >= 7')) == 0:
        return False
    else:
        return True

In [34]:
def respetaSeriacion(clave,kardex):
    if len(seriaciones.query('ser2 == "' + clave + '"')) == 0:
        return True
    else:
        #Se guarda en necesarias la lista de asignaturas necesarias para llevar la asignura "clave"
        necesarias = seriaciones.query('ser2 == "' + clave + '"')['ser1'].unique()
        
        #Se recorre la lista para comprobar si el estudiante ya pasó las asignaturas necesarias, en caso de que le falte 
        #al menos una, entonces no se respeta la seriación.
        for necesaria in necesarias:
            if not(materiaHaSidoAprobada(kardex,necesaria)):
                return False
        return True

In [35]:
#Filtra las materias que no cumplen las restricciones 1 y 2

def obtenerOfertaUtil(kardex,oferta):
    #Se eliminan las materias en la oferta que ya han sido aprobadas
    aprobadas = kardex.query('promediofinal >= 7')['clave'].unique()
    for clave in aprobadas:
        oferta = oferta.query('Clave != "' + clave + '"')
        
    #Se eliminan las materias que el alumno no puede llevar por la seriación
    ofertaUtilIndex = set(oferta.index.values)
    for idx in oferta.index:
        if not(respetaSeriacion(oferta.loc[idx]['Clave'],kardex)):
            ofertaUtilIndex.remove(idx)
            
    #Se eliminan prácticas profesionales y proyecto terminal
    for i in range(len(oferta['Clave'].values)):
        clave = oferta['Clave'].values[i]
        if clave[0:3] == 'PID' or clave == 'IT0427':
            try:
                ofertaUtilIndex.remove(oferta.index.values[i])
            except:
                continue
    ofertaUtil = oferta.loc[list(ofertaUtilIndex)]
    
    #Se eliminan materias de elección libre de primer y segundo ciclo
    if(materiaHaSidoAprobada(kardex,'IL0102')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0160"')
    if(materiaHaSidoAprobada(kardex,'ID0160')):
        ofertaUtil = ofertaUtil.query('Clave != "IL0102"')
    
    if(materiaHaSidoAprobada(kardex,'IT0103')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0161"')
    if(materiaHaSidoAprobada(kardex,'ID0161')):
        ofertaUtil = ofertaUtil.query('Clave != "IT0103"')
        
    if(materiaHaSidoAprobada(kardex,'ID0264')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0262"')
    if(materiaHaSidoAprobada(kardex,'ID0262')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0264"')
        
    if(materiaHaSidoAprobada(kardex,'ID0263')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0265"')
    if(materiaHaSidoAprobada(kardex,'ID0265')):
        ofertaUtil = ofertaUtil.query('Clave != "ID0263"')
        
    
    return ofertaUtil

## Generación de carga aleatoria válida

#### NOTA: Tomar en cuenta materias repetidas y por ahora descartar talleres deportivos, artísticos, lengua inglesa y prácticas profesionales

En esta sección creamos las funciones "generarCargaValida" la cual retorna una string de bits representando una carga académica con base a la oferta útil. También se crea la función "obtenerDatosCarga" que nos ayuda a obtener los datos de una carga académica con base a la oferta útil y a una string de bits que denominamos "bits de carga"

In [36]:
def generarSolucionAleatoria(ofertaUtil):
    solucion = [-1] * 9
    for i in range(9):
        gen = np.random.randint(-1,len(ofertaUtil))
        if gen == -1:
            solucion[i] = -1
        else:
            solucion[i] = ofertaUtil.index[gen]
    return solucion

In [37]:
def obtenerDatosCarga(solucion,ofertaUtil):
    solucionU = np.array(solucion)
    solucionU = np.unique(solucionU)
    solucionU = solucionU[solucionU >= 0]
    
    return ofertaUtil.loc[solucionU]

In [38]:
def comprobarTraslapacion(solucion,ofertaUtil):
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado']
    
    for dia in dias:
        horarioDia = datosCarga.sort_values(dia)[dia].values
        for i in range(len(horarioDia)):
            if horarioDia[i] == '-':
                continue
            horaInicioI = int(horarioDia[i][0:2])
            horaFinI = int(horarioDia[i][6:8])
            
            for j in range(i+1,len(horarioDia)):
                horaInicioJ = int(horarioDia[j][0:2])
                horaFinJ = int(horarioDia[j][6:8])
                
                if not(horaFinJ  <= horaInicioI or horaInicioJ >= horaFinI):
                    return True
    return False

In [39]:
def esValido(solucion,ofertaUtil):
    #Si se repite una materia es inválido
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    if len(datosCarga['Clave'].unique()) < len(datosCarga):
        return False
    #Si se traslapan dos materias es inválido
    if comprobarTraslapacion(solucion,ofertaUtil):
        return False
    return True

In [40]:
def generarSolucionValida(ofertaUtil):
    bitsDeCarga = generarSolucionAleatoria(ofertaUtil)
    
    while not(esValido(bitsDeCarga,ofertaUtil)):
        bitsDeCarga = generarSolucionAleatoria(ofertaUtil)
    
    return bitsDeCarga

In [41]:
def obtenerHorario(carga,ofertaUtil):
    primeraHoraMinima = 24
    ultimaHoraMaxima = 0
    datosCarga = obtenerDatosCarga(carga,ofertaUtil)
    horario = pd.DataFrame({
        'Hora': ['7:00-8:00','8:00-9:00','9:00-10:00','10:00-11:00','11:00-12:00','12:00-13:00','13:00-14:00','14:00-15:00','15:00-16:00','16:00-17:00','17:00-18:00','18:00-19:00','19:00-20:00','20:00-21:00','21:00-22:00'],
        'Lunes': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Martes': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Miercoles': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Jueves': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Viernes': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
        'Sabado': ['-','-','-','-','-','-','-','-','-','-','-','-','-','-','-'],
    })
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado']
    for dia in dias:
        for i in range(len(datosCarga)):
            if datosCarga[dia].iloc[i] == '-':
                continue

            horaInicio = int(datosCarga.iloc[i][dia][0:2])
            horaFin = int(datosCarga.iloc[i][dia][6:8])
            
            primeraHoraMinima = min(primeraHoraMinima,horaInicio)
            ultimaHoraMaxima = max(ultimaHoraMaxima,horaFin)
            
            nombre = datosCarga.iloc[i]['Nombre']

            for hora in range(horaInicio,horaFin):
                horario.loc[hora-7,dia]=nombre
    return horario[(primeraHoraMinima-7):(ultimaHoraMaxima-6)]

# Definición de funciones de utilidad y de costo

## Utilidad de carga académica con base en las materias reprobadas

In [42]:
def obtenerMateriasReprobadas(kardex):
    materiasReprobadas = kardex.query('promediofinal < 7')['clave'].unique()
    materiasReprobadasFinal = []
    for clave in materiasReprobadas:
        aprobado = len(kardex.query('clave == "' + clave + '" and promediofinal >= 7'))
        if aprobado == 0:
            materiasReprobadasFinal.append(clave)
    return materiasReprobadasFinal

In [43]:
def UpMR(kardex,solucion,ofertaUtil):
    materiasReprobadas = obtenerMateriasReprobadas(kardex)
    if len(materiasReprobadas) == 0:
        return 0
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    utilidadTotal = 0
    
    for clave in materiasReprobadas:
        cargado = len(datosCarga.query('Clave == "' + clave + '"'))
        if cargado == 1:
            utilidadTotal += 1
            
    
    #Normalización
    utilidad = (utilidadTotal)/(len(materiasReprobadas))
    return utilidad

## Utilidad de carga académica con base en el cierre de ciclos

In [44]:
def UpCC(solucion,ofertaUtil):
    alpha = 3
    utilidad = [alpha**3,alpha**2,alpha,1]
    utilidadTotal = 0
    
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    
    claves = datosCarga['Clave'].unique()
    for i in range(len(claves)):
        if claves[i][0:2] == 'AD'or claves[i][0:2] == 'TA' or claves[i][0:2] == 'LI' or claves[i][0:2] == 'PI':
            continue
            
        ciclo = int(plan_2016.query('clave == "' + claves[i] + '"')['ciclos'])
        utilidadTotal += utilidad[ciclo-1]
    
    #Normalización
    utilidadNorm = (utilidadTotal)/(utilidad[0]*9)
    return utilidadNorm

## Costo de carga académica con base en las horas libres

#### NOTA: Tomar en cuenta las horas libres por día

In [45]:
    def CpHL(solucion,ofertaUtil):
        dias = ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado']
        costoTotal = 0
        hlMax = 0

        datosCarga = obtenerDatosCarga(solucion,ofertaUtil)

        for dia in dias:
            datosCarga = datosCarga.sort_values(dia)
            if datosCarga[dia].iloc[len(datosCarga)-2] == '-':
                continue

            ultimaHoraFin = 0
            for i in range(len(datosCarga)):
                if datosCarga.iloc[i][dia] == '-':
                    continue
                if ultimaHoraFin == 0:
                    ultimaHoraFin = int(datosCarga.iloc[i][dia][6:8])
                    primeraHoraDia = int(datosCarga.iloc[i][dia][0:2])
                    continue
                horaInicio = int(datosCarga.iloc[i][dia][0:2])
                costoTotal += (horaInicio - ultimaHoraFin)
                ultimaHoraFin = int(datosCarga.iloc[i][dia][6:8])
            hlMax += (ultimaHoraFin - primeraHoraDia - 2)

        if hlMax == 0:
            return 0
        #Normalización
        costo = (costoTotal)/(hlMax)
        return costo

## Costo de carga académica con base en la disponibilidad de horario del estudiante

Creamos 4 casos diferentes donde los estudiantes trabajan. Guardamos el horario de ocupación de los estudiantes en un dataframe por cada estudiante.

In [46]:
# El estudiante trabaja por las mañanas tiempo completo
disp_est_1 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Martes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Miercoles": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Jueves": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Viernes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Sabado": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True]
})


# El estudiante trabaja por las tardes tiempo completo
disp_est_2 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Martes": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Miercoles": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Jueves": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Viernes": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False],
    "Sabado": [True,True,True,True,True,True,False,False,False,False,False,False,False,False,False]
})

# El estudiante trabaja por las mañanas medio tiempo
disp_est_3 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Martes": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Miercoles": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Jueves": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Viernes": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False],
    "Sabado": [True,False,False,False,False,False,True,True,True,True,True,True,True,False,False]
})
# El estudiante trabaja por las tardes medio tiempo
disp_est_4 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Martes": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Miercoles": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Jueves": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Viernes": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True],
    "Sabado": [True,True,True,True,True,False,False,False,False,False,True,True,True,True,True]
})
disp_est_5 = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Martes": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Miercoles": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Jueves": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Viernes": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True],
    "Sabado": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True]
})

In [47]:
disponibilidadArr = [disp_est_1,disp_est_2,disp_est_3,disp_est_4,disp_est_5]

In [48]:
def CpDH(solucion,ofertaUtil,disponibilidad):
    disponibilidadTotal = sum(disponibilidad['Lunes']) + sum(disponibilidad['Martes']) + sum(disponibilidad['Miercoles']) + sum(disponibilidad['Jueves']) + sum(disponibilidad['Viernes']) + sum(disponibilidad['Sabado'])
    if disponibilidadTotal == 90:
        return 0
    
    dias = ['Lunes','Martes','Miercoles','Jueves','Viernes','Sabado']
    datosCarga = obtenerDatosCarga(solucion,ofertaUtil)
    costoTotal = 0

    for dia in dias:
        for i in range(len(datosCarga)):
            if datosCarga[dia].iloc[i] == '-':
                continue

            horaInicio = int(datosCarga.iloc[i][dia][0:2])
            horaFin = int(datosCarga.iloc[i][dia][6:8])

            for hora in range(horaInicio,horaFin):
                if not(disponibilidad.query('hora == ' + str(hora))[dia].values[0]):
                    costoTotal += 1
                    
    #Normalización
    costo = (costoTotal)/(90 - disponibilidadTotal)
    return costo

# Definición de la función de desempeño

In [49]:
def obtenerDesempeno(solucion,cromosomaIndice):
    if not(esValido(solucion,ofertaUtil)):
        return 0
    pesos = np.array([2,1,1,1])   
    solucionlist = list(solucion)
    solucionset = set(solucionlist)
    if(len(solucionset)<3):
        return 0
    
    upcc = UpCC(solucion,ofertaUtil)
    upmr = UpMR(kardex,solucion,ofertaUtil)
    cpdh = CpDH(solucion,ofertaUtil,disponibilidad)
    cphl = CpHL(solucion,ofertaUtil)
    
    x = np.array([upcc,upmr,cpdh,cphl])
    x = x*pesos
    
    x[2] =  pesos[2]- x[2]
    x[3] = pesos[3] - x[3]
    
    return sum(x)

In [50]:
def evaluarSolucion(solucion):
    pesos = np.array([1,1,1,1])   
    
    upcc = UpCC(solucion,ofertaUtil)
    upmr = UpMR(kardex,solucion,ofertaUtil)
    cpdh = CpDH(solucion,ofertaUtil,disponibilidad)
    cphl = CpHL(solucion,ofertaUtil)
    
    print("La carga evaluada tiene:")
    print("\t" + str(round(100*upcc,2)) + "% de utilidad por las materias de primeros ciclos llevadas.")
    print("\t" + str(round(100*upmr,2)) + "% del total de las materias reprobadas del estudiante.") 
    print("\t" + str(round(100*cpdh,2)) + "% de costo por sobreposición con las horas donde el estudiante no está disponible.")
    print("\t" + str(round(100*cphl,2)) + "% de costo por el total de horas libres por día.")

### Problemas a solucionar:

-Función de horas libres no toma en cuenta el promedio de horas libres por día (Hacer otra función?)

-¿La función objetivo de regresar un promedio o una suma de las demás funciones?

-Arreglar costo por horas libres, regresa valores negativos

# Algoritmos evolutivos

## Análisis cualitativo

### Rodrigo

In [51]:
claves = ["DP0191","DP0194","ID0102","ID0160","II0106","IT0101","IT0161","DP0193","ID0103","ID0161","II0106","IT0104","IT0264","DP0295","DP0396","ID0102","II0209","II0319","II0322","IT0107","DP0498","ID0103","ID0205","ID0206","ID0265","II0320","IL0204","DP0497","ID0101","ID0264","ID0308","ID0309","ID3466","II0215","IT0425","ID0204","ID0205","ID0307","ID0414","ID3422","II0422","NI3431","DP0499","ID0413","IL0414","IT0210","IT0322"]
periodos = [201903,201903,201903,201903,201903,201903,201903,202001,202001,202001,202001,202001,202001,202003,202003,202003,202003,202003,202003,202003,202101,202101,202101,202101,202101,202101,202101,202103,202103,202103,202103,202103,202103,202103,202103,202201,202201,202201,202201,202201,202201,202201,202203,202203,202203,202203,202203]
promedios = [9,8,5,9,5,8,8,8,0,9,7,8,9,9,10,7,7,8,10,9,10,9,5,9,10,9,9,9,9,10,7,7,9,7,3,7,8,9,8,8,10,9,8,8,10,8,9]
carrera = ['Ingeniería en Datos e Inteligencia Organizacional'] * len(claves)
matricula = [190300432] * len(claves)

In [52]:
kardex = pd.DataFrame({
    'clave': claves,
    'periodo': periodos,
    'carrera': carrera,
    'n_matricula': matricula,
    'promediofinal': promedios
})

In [53]:
kardex = pd.read_csv('./kardex/OROPEZA_RODRIGO_KARDEX.csv')

In [54]:
for clave in obtenerMateriasReprobadas(kardex):
    print(clave,plan_2016.query('clave == "' + clave + '"')['nombre'].values[0])

IT0425 Introducción a la inteligencia artificial


In [55]:
obtenerCreditos(kardex)

272

In [56]:
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [57]:
disponibilidad = disponibilidadArr[4]

In [58]:
instancia1 = pg.GA(num_generations=20,
                   num_parents_mating=2,
                   fitness_func=obtenerDesempeno,
                   sol_per_pop=500,
                   num_genes=9,
                   parent_selection_type="sss",
                   keep_parents=1,
                   crossover_type='uniform',
                   mutation_type='random',
                   mutation_percent_genes=15,
                   gene_type=int,
                   gene_space=np.append(ofertaUtil.index.values,[-1]))

In [59]:
start_time = time.time()
instancia1.run()
tiempoejecucion = time.time() - start_time
print('Proceso terminado en ' + str(tiempoejecucion))

Proceso terminado en 147.7632987499237


In [60]:
recomendaciones = instancia1.population[instancia1.last_generation_fitness != 0].copy()
recomendacionesFinal = []
for rec in recomendaciones:
    recomendacionesFinal.append(list(set(rec)))
recomendacionesFinal = list(np.unique(recomendacionesFinal))
recomendacionesFinal.sort(key=lambda x: -obtenerDesempeno(x,0))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (336,) + inhomogeneous part.

#### Resultados

##### #1

In [61]:
obtenerDatosCarga(recomendacionesFinal[0],ofertaUtil)

,Clave,Nombre,Maestro,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
58,IL3422,Liderazgo y equipos de trabajo,García Rivera / Miriam Angelica,-,-,15:00-16:00,-,14:00-16:00,-
69,ID3416,Diseño multimedia,Landeros Alvarado / Aurea,-,08:00-09:00,-,09:00-11:00,-,-
75,ID0310,Minería de datos,Gómez García / Hector Fernando,-,-,16:00-18:00,-,16:00-18:00,-
79,IT0263,Electrónica analógica,Márquez Chacón / Wilberth,-,-,18:00-20:00,-,18:00-19:00,-
81,IT0425,Introducción a la inteligencia artificial,Sansores Pérez / Candelaria Elizabeth,14:00-16:00,-,14:00-15:00,-,-,-


In [62]:
obtenerHorario(recomendacionesFinal[0],ofertaUtil)

,Hora,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
1,8:00-9:00,-,Diseño multimedia,-,-,-,-
2,9:00-10:00,-,-,-,Diseño multimedia,-,-
3,10:00-11:00,-,-,-,Diseño multimedia,-,-
4,11:00-12:00,-,-,-,-,-,-
5,12:00-13:00,-,-,-,-,-,-
6,13:00-14:00,-,-,-,-,-,-
7,14:00-15:00,Introducción a la inteligencia artificial,-,Introducción a la inteligencia artificial,-,Liderazgo y equipos de trabajo,-
8,15:00-16:00,Introducción a la inteligencia artificial,-,Liderazgo y equipos de trabajo,-,Liderazgo y equipos de trabajo,-
9,16:00-17:00,-,-,Minería de datos,-,Minería de datos,-
10,17:00-18:00,-,-,Minería de datos,-,Minería de datos,-


In [63]:
evaluarSolucion(recomendacionesFinal[0])

La carga evaluada tiene:
	5.35% de utilidad por las materias de primeros ciclos llevadas.
	100.0% del total de las materias reprobadas del estudiante.
	0% de costo por sobreposición con las horas donde el estudiante no está disponible.
	0.0% de costo por el total de horas libres por día.


### Karina

In [217]:
kardex = pd.read_csv('./kardex/MIRANDA_KARINA_KARDEX.csv')
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [218]:
for clave in obtenerMateriasReprobadas(kardex):
    print(clave,plan_2016.query('clave == "' + clave + '"')['nombre'].values[0])
print('\nCreditos: ' + str(obtenerCreditos(kardex)))


Creditos: 202


In [219]:
disponibilidad = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [False,False,True,True,True,True,True,True,False,False,False,False,False,False,False],
    "Martes": [False,False,False,False,True,True,True,True,True,True,False,False,False,False,False],
    "Miercoles": [False,False,True,True,True,True,True,True,False,False,False,False,False,False,False],
    "Jueves": [False,False,False,False,True,True,True,True,True,True,False,False,False,False,False],
    "Viernes": [False,False,True,True,True,True,True,True,False,False,False,False,False,False,False],
    "Sabado": [False,False,False,False,False,False,False,False,False,False,False,False,False,False,False]
})

In [220]:
instancia1 = pg.GA(num_generations=20,
                   num_parents_mating=2,
                   fitness_func=obtenerDesempeno,
                   sol_per_pop=500,
                   num_genes=9,
                   parent_selection_type="sss",
                   keep_parents=1,
                   crossover_type='uniform',
                   mutation_type='random',
                   mutation_percent_genes=15,
                   gene_type=int,
                   gene_space=np.append(ofertaUtil.index.values,[-1]))
start_time = time.time()
instancia1.run()
tiempoejecucion = time.time() - start_time
print('Proceso terminado en ' + str(tiempoejecucion))
solution, solution_fitness, solution_idx = instancia1.best_solution()

Proceso terminado en 262.235235452652


In [221]:
obtenerDatosCarga(solution,ofertaUtil)

,Clave,Nombre,Maestro,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
7,IT0104,Matemáticas discretas,Naredo García / Enrique,-,-,15:00-17:00,-,14:00-15:00,-
35,ID0206,Interconectividad en redes,Sansores Pérez / Candelaria Elizabeth,11:00-13:00,-,11:00-13:00,-,-,-
49,ID0205,Geometría computacional,Naredo García / Enrique,-,14:00-16:00,-,14:00-15:00,-,-
70,ID3468,Herramientas para la gestión de grandes volúme...,Mendiola Fuentes / Jessica Carmin,09:00-11:00,-,-,13:00-14:00,-,-
81,IT0425,Introducción a la inteligencia artificial,Sansores Pérez / Candelaria Elizabeth,14:00-16:00,-,14:00-15:00,-,-,-


In [222]:
obtenerHorario(solution,ofertaUtil)

,Hora,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
2,9:00-10:00,Herramientas para la gestión de grandes volúme...,-,-,-,-,-
3,10:00-11:00,Herramientas para la gestión de grandes volúme...,-,-,-,-,-
4,11:00-12:00,Interconectividad en redes,-,Interconectividad en redes,-,-,-
5,12:00-13:00,Interconectividad en redes,-,Interconectividad en redes,-,-,-
6,13:00-14:00,-,-,-,Herramientas para la gestión de grandes volúme...,-,-
7,14:00-15:00,Introducción a la inteligencia artificial,Geometría computacional,Introducción a la inteligencia artificial,Geometría computacional,Matemáticas discretas,-
8,15:00-16:00,Introducción a la inteligencia artificial,Geometría computacional,Matemáticas discretas,-,-,-
9,16:00-17:00,-,-,Matemáticas discretas,-,-,-
10,17:00-18:00,-,-,-,-,-,-


In [223]:
evaluarSolucion(solution)

La carga evaluada tiene:
	20.16% de utilidad por las materias de primeros ciclos llevadas.
	0% del total de las materias reprobadas del estudiante.
	5.0% de costo por sobreposición con las horas donde el estudiante no está disponible.
	22.22% de costo por el total de horas libres por día.


### Dylan

In [99]:
kardex = pd.read_csv('./kardex/CHAN_DYLAN_KARDEX.csv')
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [100]:
for clave in obtenerMateriasReprobadas(kardex):
    print(clave,plan_2016.query('clave == "' + clave + '"')['nombre'].values[0])
print('\nCreditos: ' + str(obtenerCreditos(kardex)))

IT0208 Electrónica digital
IT0425 Introducción a la inteligencia artificial

Creditos: 214


In [101]:
disponibilidad = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Martes": [True,True,False,False,False,False,False,False,True,True,True,True,True,True,True],
    "Miercoles": [True,False,False,False,False,False,True,True,True,True,True,True,True,True,True],
    "Jueves": [True,False,False,False,False,False,False,False,True,True,True,True,True,True,True],
    "Viernes": [True,False,False,False,False,True,True,True,True,True,True,True,True,True,True],
    "Sabado": [True,True,True,True,True,True,True,True,True,True,True,True,True,True,True]
})

In [102]:
instancia1 = pg.GA(num_generations=20,
                   num_parents_mating=2,
                   fitness_func=obtenerDesempeno,
                   sol_per_pop=500,
                   num_genes=9,
                   parent_selection_type="sss",
                   keep_parents=1,
                   crossover_type='uniform',
                   mutation_type='random',
                   mutation_percent_genes=15,
                   gene_type=int,
                   gene_space=np.append(ofertaUtil.index.values,[-1]))
start_time = time.time()
instancia1.run()
tiempoejecucion = time.time() - start_time
print('Proceso terminado en ' + str(tiempoejecucion))
solution, solution_fitness, solution_idx = instancia1.best_solution()

Proceso terminado en 594.8886957168579


In [103]:
obtenerDatosCarga(solution,ofertaUtil)

,Clave,Nombre,Maestro,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
42,ID0205,Geometría computacional,Naredo García / Enrique,-,20:00-21:00,-,-,18:00-20:00,-
75,ID0310,Minería de datos,Gómez García / Hector Fernando,-,-,16:00-18:00,-,16:00-18:00,-
81,IT0425,Introducción a la inteligencia artificial,Sansores Pérez / Candelaria Elizabeth,14:00-16:00,-,14:00-15:00,-,-,-
83,IT0317,Diseño de interfaces humano-máquina,Morales Saavedra / Emmanuel,-,-,15:00-16:00,-,14:00-16:00,-
94,ID0414,Seguridad de datos,Aguilar Pérez / Jairo Cesar,-,18:00-20:00,-,18:00-20:00,-,-
97,ID3423,Sistemas de apoyo para la toma de decisiones,Mendiola Fuentes / Jessica Carmin,-,16:00-18:00,-,17:00-18:00,-,-


In [105]:
evaluarSolucion(solution)

La carga evaluada tiene:
	7.41% de utilidad por las materias de primeros ciclos llevadas.
	50.0% del total de las materias reprobadas del estudiante.
	6.45% de costo por sobreposición con las horas donde el estudiante no está disponible.
	0.0% de costo por el total de horas libres por día.


In [106]:
obtenerHorario(solution,ofertaUtil)

,Hora,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
7,14:00-15:00,Introducción a la inteligencia artificial,-,Introducción a la inteligencia artificial,-,Diseño de interfaces humano-máquina,-
8,15:00-16:00,Introducción a la inteligencia artificial,-,Diseño de interfaces humano-máquina,-,Diseño de interfaces humano-máquina,-
9,16:00-17:00,-,Sistemas de apoyo para la toma de decisiones,Minería de datos,-,Minería de datos,-
10,17:00-18:00,-,Sistemas de apoyo para la toma de decisiones,Minería de datos,Sistemas de apoyo para la toma de decisiones,Minería de datos,-
11,18:00-19:00,-,Seguridad de datos,-,Seguridad de datos,Geometría computacional,-
12,19:00-20:00,-,Seguridad de datos,-,Seguridad de datos,Geometría computacional,-
13,20:00-21:00,-,Geometría computacional,-,-,-,-
14,21:00-22:00,-,-,-,-,-,-


### Miguel

In [74]:
kardex = pd.read_csv('./kardex/CANDANEDO_JASMIN_KARDEX.csv')
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [75]:
for clave in obtenerMateriasReprobadas(kardex):
    print(clave,plan_2016.query('clave == "' + clave + '"')['nombre'].values[0])
print('\nCreditos: ' + str(obtenerCreditos(kardex)))

ID0205 Geometría computacional
ID0413 Procesamiento de datos en la nube

Creditos: 282


In [76]:
disponibilidad = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Martes": [False,False,False,False,False,False,False,True,True,True,True,True,True,True,True],
    "Miercoles": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Jueves": [False,False,False,False,False,False,False,True,True,True,True,True,True,True,True],
    "Viernes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Sabado": [False,False,False,False,False,False,False,False,False,False,False,False,False,False,False]
})

In [77]:
instancia1 = pg.GA(num_generations=20,
                   num_parents_mating=2,
                   fitness_func=obtenerDesempeno,
                   sol_per_pop=500,
                   num_genes=9,
                   parent_selection_type="sss",
                   keep_parents=1,
                   crossover_type='uniform',
                   mutation_type='random',
                   mutation_percent_genes=15,
                   gene_type=int,
                   gene_space=np.append(ofertaUtil.index.values,[-1]))
start_time = time.time()
instancia1.run()
tiempoejecucion = time.time() - start_time
print('Proceso terminado en ' + str(tiempoejecucion))
solution, solution_fitness, solution_idx = instancia1.best_solution()

Proceso terminado en 379.33377981185913


In [78]:
obtenerDatosCarga(solution,ofertaUtil)

,Clave,Nombre,Maestro,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
43,ID0206,Interconectividad en redes,Sansores Pérez / Candelaria Elizabeth,-,18:00-20:00,-,16:00-18:00,-,-
49,ID0205,Geometría computacional,Naredo García / Enrique,-,14:00-16:00,-,14:00-15:00,-,-
92,II0423,Métodos numéricos por computadora,Mejenes Sánchez / Omar,-,16:00-18:00,18:00-20:00,-,-,-
98,IT0423,Administración de proyectos de desarrollo de s...,Galvan Herrera / Maria Fernanda,19:00-21:00,-,-,20:00-21:00,-,-
100,IT3472,Principios de automatización y robótica,Landaverde Lorenzo / Pablo Jacob,-,20:00-22:00,-,21:00-22:00,-,-
101,NI3431,Marketing en los negocios electrónicos,Canche Dzul / Arely Del Socorro,21:00-22:00,-,-,-,19:00-21:00,-


In [79]:
evaluarSolucion(solution)

La carga evaluada tiene:
	9.05% de utilidad por las materias de primeros ciclos llevadas.
	50.0% del total de las materias reprobadas del estudiante.
	0.0% de costo por sobreposición con las horas donde el estudiante no está disponible.
	23.08% de costo por el total de horas libres por día.


In [80]:
obtenerHorario(solution,ofertaUtil)

,Hora,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
7,14:00-15:00,-,Geometría computacional,-,Geometría computacional,-,-
8,15:00-16:00,-,Geometría computacional,-,-,-,-
9,16:00-17:00,-,Métodos numéricos por computadora,-,Interconectividad en redes,-,-
10,17:00-18:00,-,Métodos numéricos por computadora,-,Interconectividad en redes,-,-
11,18:00-19:00,-,Interconectividad en redes,Métodos numéricos por computadora,-,-,-
12,19:00-20:00,Administración de proyectos de desarrollo de s...,Interconectividad en redes,Métodos numéricos por computadora,-,Marketing en los negocios electrónicos,-
13,20:00-21:00,Administración de proyectos de desarrollo de s...,Principios de automatización y robótica,-,Administración de proyectos de desarrollo de s...,Marketing en los negocios electrónicos,-
14,21:00-22:00,Marketing en los negocios electrónicos,Principios de automatización y robótica,-,Principios de automatización y robótica,-,-


### Alan

In [122]:
kardex = pd.read_csv('./kardex/CENTENO_ALAN_KARDEX.csv')
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [123]:
for clave in obtenerMateriasReprobadas(kardex):
    print(clave,plan_2016.query('clave == "' + clave + '"')['nombre'].values[0])
print('\nCreditos: ' + str(obtenerCreditos(kardex)))

IT0425 Introducción a la inteligencia artificial
ID0205 Geometría computacional

Creditos: 282


In [124]:
disponibilidad = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [False,False,True,True,True,True,True,False,False,False,False,False,False,False,False],
    "Martes": [False,False,True,True,True,True,True,False,False,False,False,False,False,False,False],
    "Miercoles": [False,False,True,True,True,True,True,False,False,False,False,False,False,False,False],
    "Jueves": [False,False,True,True,True,True,True,False,False,False,False,False,False,False,False],
    "Viernes": [False,False,True,True,True,True,True,False,False,False,False,False,False,False,False],
    "Sabado": [False,True,True,True,False,False,False,False,False,False,False,False,False,False,False]
})

In [125]:
instancia1 = pg.GA(num_generations=20,
                   num_parents_mating=2,
                   fitness_func=obtenerDesempeno,
                   sol_per_pop=500,
                   num_genes=9,
                   parent_selection_type="sss",
                   keep_parents=1,
                   crossover_type='uniform',
                   mutation_type='random',
                   mutation_percent_genes=15,
                   gene_type=int,
                   gene_space=np.append(ofertaUtil.index.values,[-1]))
start_time = time.time()
instancia1.run()
tiempoejecucion = time.time() - start_time
print('Proceso terminado en ' + str(tiempoejecucion))
solution, solution_fitness, solution_idx = instancia1.best_solution()

Proceso terminado en 251.61857318878174


In [126]:
obtenerDatosCarga(solution,ofertaUtil)

,Clave,Nombre,Maestro,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
42,ID0205,Geometría computacional,Naredo García / Enrique,-,20:00-21:00,-,-,18:00-20:00,-
75,ID0310,Minería de datos,Gómez García / Hector Fernando,-,-,16:00-18:00,-,16:00-18:00,-
81,IT0425,Introducción a la inteligencia artificial,Sansores Pérez / Candelaria Elizabeth,14:00-16:00,-,14:00-15:00,-,-,-
83,IT0317,Diseño de interfaces humano-máquina,Morales Saavedra / Emmanuel,-,-,15:00-16:00,-,14:00-16:00,-


In [127]:
evaluarSolucion(solution)

La carga evaluada tiene:
	6.58% de utilidad por las materias de primeros ciclos llevadas.
	100.0% del total de las materias reprobadas del estudiante.
	20.97% de costo por sobreposición con las horas donde el estudiante no está disponible.
	0.0% de costo por el total de horas libres por día.


In [128]:
obtenerHorario(solution,ofertaUtil)

,Hora,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
7,14:00-15:00,Introducción a la inteligencia artificial,-,Introducción a la inteligencia artificial,-,Diseño de interfaces humano-máquina,-
8,15:00-16:00,Introducción a la inteligencia artificial,-,Diseño de interfaces humano-máquina,-,Diseño de interfaces humano-máquina,-
9,16:00-17:00,-,-,Minería de datos,-,Minería de datos,-
10,17:00-18:00,-,-,Minería de datos,-,Minería de datos,-
11,18:00-19:00,-,-,-,-,Geometría computacional,-
12,19:00-20:00,-,-,-,-,Geometría computacional,-
13,20:00-21:00,-,Geometría computacional,-,-,-,-
14,21:00-22:00,-,-,-,-,-,-


### Jasmin

In [106]:
kardex = pd.read_csv('./kardex/CANDANEDO_JASMIN_KARDEX.csv')
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [107]:
for clave in obtenerMateriasReprobadas(kardex):
    print(clave,plan_2016.query('clave == "' + clave + '"')['nombre'].values[0])
print('\nCreditos: ' + str(obtenerCreditos(kardex)))

ID0205 Geometría computacional

Creditos: 278


In [108]:
disponibilidad = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Martes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Miercoles": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Jueves": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Viernes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Sabado": [False,False,False,False,False,False,False,False,False,False,False,False,False,False,False]
})

In [109]:
instancia1 = pg.GA(num_generations=20,
                   num_parents_mating=2,
                   fitness_func=obtenerDesempeno,
                   sol_per_pop=500,
                   num_genes=9,
                   parent_selection_type="sss",
                   keep_parents=1,
                   crossover_type='uniform',
                   mutation_type='random',
                   mutation_percent_genes=15,
                   gene_type=int,
                   gene_space=np.append(ofertaUtil.index.values,[-1]))
start_time = time.time()
instancia1.run()
tiempoejecucion = time.time() - start_time
print('Proceso terminado en ' + str(tiempoejecucion))
solution, solution_fitness, solution_idx = instancia1.best_solution()

Proceso terminado en 278.36793303489685


In [111]:
obtenerDatosCarga(solution,ofertaUtil)

,Clave,Nombre,Maestro,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
49,ID0205,Geometría computacional,Naredo García / Enrique,-,14:00-16:00,-,14:00-15:00,-,-
75,ID0310,Minería de datos,Gómez García / Hector Fernando,-,-,16:00-18:00,-,16:00-18:00,-
94,ID0414,Seguridad de datos,Aguilar Pérez / Jairo Cesar,-,18:00-20:00,-,18:00-20:00,-,-
96,ID3420,Analítica para la inteligencia de negocios,Gómez García / Hector Fernando,-,-,18:00-20:00,-,18:00-19:00,-
97,ID3423,Sistemas de apoyo para la toma de decisiones,Mendiola Fuentes / Jessica Carmin,-,16:00-18:00,-,17:00-18:00,-,-
98,IT0423,Administración de proyectos de desarrollo de s...,Galvan Herrera / Maria Fernanda,19:00-21:00,-,-,20:00-21:00,-,-
100,IT3472,Principios de automatización y robótica,Landaverde Lorenzo / Pablo Jacob,-,20:00-22:00,-,21:00-22:00,-,-
101,NI3431,Marketing en los negocios electrónicos,Canche Dzul / Arely Del Socorro,21:00-22:00,-,-,-,19:00-21:00,-


In [110]:
evaluarSolucion(solution)

La carga evaluada tiene:
	7.41% de utilidad por las materias de primeros ciclos llevadas.
	100.0% del total de las materias reprobadas del estudiante.
	5.0% de costo por sobreposición con las horas donde el estudiante no está disponible.
	11.11% de costo por el total de horas libres por día.


In [112]:
obtenerHorario(solution,ofertaUtil)

,Hora,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
7,14:00-15:00,-,Geometría computacional,-,Geometría computacional,-,-
8,15:00-16:00,-,Geometría computacional,-,-,-,-
9,16:00-17:00,-,Sistemas de apoyo para la toma de decisiones,Minería de datos,-,Minería de datos,-
10,17:00-18:00,-,Sistemas de apoyo para la toma de decisiones,Minería de datos,Sistemas de apoyo para la toma de decisiones,Minería de datos,-
11,18:00-19:00,-,Seguridad de datos,Analítica para la inteligencia de negocios,Seguridad de datos,Analítica para la inteligencia de negocios,-
12,19:00-20:00,Administración de proyectos de desarrollo de s...,Seguridad de datos,Analítica para la inteligencia de negocios,Seguridad de datos,Marketing en los negocios electrónicos,-
13,20:00-21:00,Administración de proyectos de desarrollo de s...,Principios de automatización y robótica,-,Administración de proyectos de desarrollo de s...,Marketing en los negocios electrónicos,-
14,21:00-22:00,Marketing en los negocios electrónicos,Principios de automatización y robótica,-,Principios de automatización y robótica,-,-


### Darwin

In [129]:
kardex = pd.read_csv('./kardex/CAB_DARWIN_KARDEX.csv')
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [130]:
for clave in obtenerMateriasReprobadas(kardex):
    print(clave,plan_2016.query('clave == "' + clave + '"')['nombre'].values[0])
print('\nCreditos: ' + str(obtenerCreditos(kardex)))

ID0205 Geometría computacional

Creditos: 276


In [131]:
disponibilidad = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [False,False,False,False,False,False,False,True,True,True,True,True,True,True,True],
    "Martes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Miercoles": [False,False,False,False,False,False,False,True,True,True,True,True,True,True,True],
    "Jueves": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Viernes": [False,False,False,False,False,False,False,True,True,True,True,True,True,True,True],
    "Sabado": [False,False,False,False,False,False,False,False,False,False,False,False,False,False,False]
})

In [132]:
instancia1 = pg.GA(num_generations=20,
                   num_parents_mating=2,
                   fitness_func=obtenerDesempeno,
                   sol_per_pop=500,
                   num_genes=9,
                   parent_selection_type="sss",
                   keep_parents=1,
                   crossover_type='uniform',
                   mutation_type='random',
                   mutation_percent_genes=15,
                   gene_type=int,
                   gene_space=np.append(ofertaUtil.index.values,[-1]))
start_time = time.time()
instancia1.run()
tiempoejecucion = time.time() - start_time
print('Proceso terminado en ' + str(tiempoejecucion))
solution, solution_fitness, solution_idx = instancia1.best_solution()

Proceso terminado en 265.4097623825073


In [133]:
obtenerDatosCarga(solution,ofertaUtil)

,Clave,Nombre,Maestro,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
43,ID0206,Interconectividad en redes,Sansores Pérez / Candelaria Elizabeth,-,18:00-20:00,-,16:00-18:00,-,-
49,ID0205,Geometría computacional,Naredo García / Enrique,-,14:00-16:00,-,14:00-15:00,-,-
58,IL3422,Liderazgo y equipos de trabajo,García Rivera / Miriam Angelica,-,-,15:00-16:00,-,14:00-16:00,-
81,IT0425,Introducción a la inteligencia artificial,Sansores Pérez / Candelaria Elizabeth,14:00-16:00,-,14:00-15:00,-,-,-
92,II0423,Métodos numéricos por computadora,Mejenes Sánchez / Omar,-,16:00-18:00,18:00-20:00,-,-,-


In [134]:
evaluarSolucion(solution)

La carga evaluada tiene:
	9.47% de utilidad por las materias de primeros ciclos llevadas.
	100.0% del total de las materias reprobadas del estudiante.
	5.56% de costo por sobreposición con las horas donde el estudiante no está disponible.
	30.0% de costo por el total de horas libres por día.


In [135]:
obtenerHorario(solution,ofertaUtil)

,Hora,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
7,14:00-15:00,Introducción a la inteligencia artificial,Geometría computacional,Introducción a la inteligencia artificial,Geometría computacional,Liderazgo y equipos de trabajo,-
8,15:00-16:00,Introducción a la inteligencia artificial,Geometría computacional,Liderazgo y equipos de trabajo,-,Liderazgo y equipos de trabajo,-
9,16:00-17:00,-,Métodos numéricos por computadora,-,Interconectividad en redes,-,-
10,17:00-18:00,-,Métodos numéricos por computadora,-,Interconectividad en redes,-,-
11,18:00-19:00,-,Interconectividad en redes,Métodos numéricos por computadora,-,-,-
12,19:00-20:00,-,Interconectividad en redes,Métodos numéricos por computadora,-,-,-
13,20:00-21:00,-,-,-,-,-,-


### Brandon pendiente

In [138]:
kardex = pd.read_csv('./kardex/GONZALES_BRANDON_KARDEX.csv')
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [139]:
for clave in obtenerMateriasReprobadas(kardex):
    print(clave,plan_2016.query('clave == "' + clave + '"')['nombre'].values[0])
print('\nCreditos: ' + str(obtenerCreditos(kardex)))


Creditos: 260


In [ ]:
disponibilidad = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [False,False,False,False,False,False,False,True,True,True,True,True,True,True,True],
    "Martes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Miercoles": [False,False,False,False,False,False,False,True,True,True,True,True,True,True,True],
    "Jueves": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Viernes": [False,False,False,False,False,False,False,True,True,True,True,True,True,True,True],
    "Sabado": [False,False,False,False,False,False,False,False,False,False,False,False,False,False,False]
})

In [ ]:
instancia1 = pg.GA(num_generations=20,
                   num_parents_mating=2,
                   fitness_func=obtenerDesempeno,
                   sol_per_pop=500,
                   num_genes=9,
                   parent_selection_type="sss",
                   keep_parents=1,
                   crossover_type='uniform',
                   mutation_type='random',
                   mutation_percent_genes=15,
                   gene_type=int,
                   gene_space=np.append(ofertaUtil.index.values,[-1]))
start_time = time.time()
instancia1.run()
tiempoejecucion = time.time() - start_time
print('Proceso terminado en ' + str(tiempoejecucion))
solution, solution_fitness, solution_idx = instancia1.best_solution()

Proceso terminado en 225.18550443649292


In [ ]:
obtenerDatosCarga(solution,ofertaUtil)

,Clave,Nombre,Maestro,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
49,ID0205,Geometría computacional,Naredo García / Enrique,-,14:00-16:00,-,14:00-15:00,-,-
70,ID3468,Herramientas para la gestión de grandes volúme...,Mendiola Fuentes / Jessica Carmin,09:00-11:00,-,-,13:00-14:00,-,-
79,IT0263,Electrónica analógica,Márquez Chacón / Wilberth,-,-,18:00-20:00,-,18:00-19:00,-


In [ ]:
evaluarSolucion(solution)

La carga evaluada tiene:
	5.35% de utilidad por las materias de primeros ciclos llevadas.
	100.0% del total de las materias reprobadas del estudiante.
	11.11% de costo por sobreposición con las horas donde el estudiante no está disponible.
	0% de costo por el total de horas libres por día.


In [ ]:
obtenerHorario(solution,ofertaUtil)

,Hora,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
2,9:00-10:00,Herramientas para la gestión de grandes volúme...,-,-,-,-,-
3,10:00-11:00,Herramientas para la gestión de grandes volúme...,-,-,-,-,-
4,11:00-12:00,-,-,-,-,-,-
5,12:00-13:00,-,-,-,-,-,-
6,13:00-14:00,-,-,-,Herramientas para la gestión de grandes volúme...,-,-
7,14:00-15:00,-,Geometría computacional,-,Geometría computacional,-,-
8,15:00-16:00,-,Geometría computacional,-,-,-,-
9,16:00-17:00,-,-,-,-,-,-
10,17:00-18:00,-,-,-,-,-,-
11,18:00-19:00,-,-,Electrónica analógica,-,Electrónica analógica,-


### Gilberto

In [307]:
kardex = pd.read_csv('./kardex/QUIJANO_GILBERTO_KARDEX.csv')
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [308]:
for clave in obtenerMateriasReprobadas(kardex):
    print(clave,plan_2016.query('clave == "' + clave + '"')['nombre'].values[0])
print('\nCreditos: ' + str(obtenerCreditos(kardex)))

ID0205 Geometría computacional

Creditos: 320


In [309]:
disponibilidad = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Martes": [False,False,False,False,False,False,False,True,True,True,True,True,True,True,True],
    "Miercoles": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Jueves": [False,False,False,False,False,False,False,True,True,True,True,True,True,True,True],
    "Viernes": [False,False,False,False,False,False,False,False,False,True,True,True,True,True,True],
    "Sabado": [False,False,False,False,False,False,False,False,False,False,False,False,False,False,False]
})

In [310]:
instancia1 = pg.GA(num_generations=20,
                   num_parents_mating=2,
                   fitness_func=obtenerDesempeno,
                   sol_per_pop=500,
                   num_genes=9,
                   parent_selection_type="sss",
                   keep_parents=1,
                   crossover_type='uniform',
                   mutation_type='random',
                   mutation_percent_genes=15,
                   gene_type=int,
                   gene_space=np.append(ofertaUtil.index.values,[-1]))
start_time = time.time()
instancia1.run()
tiempoejecucion = time.time() - start_time
print('Proceso terminado en ' + str(tiempoejecucion))
solution, solution_fitness, solution_idx = instancia1.best_solution()

Proceso terminado en 202.90170335769653


In [311]:
obtenerDatosCarga(solution,ofertaUtil)

,Clave,Nombre,Maestro,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
42,ID0205,Geometría computacional,Naredo García / Enrique,-,20:00-21:00,-,-,18:00-20:00,-
43,ID0206,Interconectividad en redes,Sansores Pérez / Candelaria Elizabeth,-,18:00-20:00,-,16:00-18:00,-,-
92,II0423,Métodos numéricos por computadora,Mejenes Sánchez / Omar,-,16:00-18:00,18:00-20:00,-,-,-
93,IL0414,Taller de argumentación y técnicas de negociación,Contreras Macossay / Bani Obed,19:00-20:00,-,20:00-22:00,-,-,-


In [312]:
evaluarSolucion(solution)

La carga evaluada tiene:
	8.23% de utilidad por las materias de primeros ciclos llevadas.
	100.0% del total de las materias reprobadas del estudiante.
	0.0% de costo por sobreposición con las horas donde el estudiante no está disponible.
	0.0% de costo por el total de horas libres por día.


In [313]:
obtenerHorario(solution,ofertaUtil)

,Hora,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
9,16:00-17:00,-,Métodos numéricos por computadora,-,Interconectividad en redes,-,-
10,17:00-18:00,-,Métodos numéricos por computadora,-,Interconectividad en redes,-,-
11,18:00-19:00,-,Interconectividad en redes,Métodos numéricos por computadora,-,Geometría computacional,-
12,19:00-20:00,Taller de argumentación y técnicas de negociación,Interconectividad en redes,Métodos numéricos por computadora,-,Geometría computacional,-
13,20:00-21:00,-,Geometría computacional,Taller de argumentación y técnicas de negociación,-,-,-
14,21:00-22:00,-,-,Taller de argumentación y técnicas de negociación,-,-,-


### Ivan

In [51]:
kardex = pd.read_csv('./kardex/RODRIGUEZ_IVAN_KARDEX.csv')
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [52]:
for clave in obtenerMateriasReprobadas(kardex):
    print(clave,plan_2016.query('clave == "' + clave + '"')['nombre'].values[0])
print('\nCreditos: ' + str(obtenerCreditos(kardex)))

IT0425 Introducción a la inteligencia artificial

Creditos: 284


In [53]:
disponibilidad = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,True,True,True,True,True,True,True,True,True,True,False,False,False,False],
    "Martes": [True,True,True,True,True,True,True,True,True,True,True,False,False,False,False],
    "Miercoles": [True,True,True,True,True,True,True,True,True,True,True,False,False,False,False],
    "Jueves": [True,True,True,True,True,True,True,True,True,True,True,False,False,False,False],
    "Viernes": [True,True,True,True,True,True,True,True,True,True,True,False,False,False,False],
    "Sabado": [False,False,False,False,False,False,False,False,False,False,False,False,False,False,False]
})

In [54]:
instancia1 = pg.GA(num_generations=20,
                   num_parents_mating=2,
                   fitness_func=obtenerDesempeno,
                   sol_per_pop=500,
                   num_genes=9,
                   parent_selection_type="sss",
                   keep_parents=1,
                   crossover_type='uniform',
                   mutation_type='random',
                   mutation_percent_genes=15,
                   gene_type=int,
                   gene_space=np.append(ofertaUtil.index.values,[-1]))
start_time = time.time()
instancia1.run()
tiempoejecucion = time.time() - start_time
print('Proceso terminado en ' + str(tiempoejecucion))
solution, solution_fitness, solution_idx = instancia1.best_solution()

Proceso terminado en 215.0476472377777


In [55]:
obtenerDatosCarga(solution,ofertaUtil)

,Clave,Nombre,Maestro,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
74,ID0307,Ingeniería de software,Aguas García / Nancy,-,16:00-18:00,-,16:00-17:00,-,-
75,ID0310,Minería de datos,Gómez García / Hector Fernando,-,-,16:00-18:00,-,16:00-18:00,-
81,IT0425,Introducción a la inteligencia artificial,Sansores Pérez / Candelaria Elizabeth,14:00-16:00,-,14:00-15:00,-,-,-
83,IT0317,Diseño de interfaces humano-máquina,Morales Saavedra / Emmanuel,-,-,15:00-16:00,-,14:00-16:00,-
102,DP0498,Problemas de México,Díaz Molina / Libertad Fidelina,-,14:00-16:00,-,15:00-16:00,-,-


In [58]:
evaluarSolucion(solution)

La carga evaluada tiene:
	5.35% de utilidad por las materias de primeros ciclos llevadas.
	100.0% del total de las materias reprobadas del estudiante.
	0.0% de costo por sobreposición con las horas donde el estudiante no está disponible.
	0.0% de costo por el total de horas libres por día.


In [59]:
obtenerHorario(solution,ofertaUtil)

,Hora,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
7,14:00-15:00,Introducción a la inteligencia artificial,Problemas de México,Introducción a la inteligencia artificial,-,Diseño de interfaces humano-máquina,-
8,15:00-16:00,Introducción a la inteligencia artificial,Problemas de México,Diseño de interfaces humano-máquina,Problemas de México,Diseño de interfaces humano-máquina,-
9,16:00-17:00,-,Ingeniería de software,Minería de datos,Ingeniería de software,Minería de datos,-
10,17:00-18:00,-,Ingeniería de software,Minería de datos,-,Minería de datos,-
11,18:00-19:00,-,-,-,-,-,-


### prueba

In [55]:
kardex = obtenerKardex(190311499).query('periodo < 202101')
ofertaUtil = obtenerOfertaUtil(kardex,oferta)

In [52]:
for clave in obtenerMateriasReprobadas(kardex):
    print(clave,plan_2016.query('clave == "' + clave + '"')['nombre'].values[0])
print('\nCreditos: ' + str(obtenerCreditos(kardex)))


Creditos: 146


In [53]:
disponibilidad = pd.DataFrame({
    "hora": [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],
    "Lunes": [True,True,True,True,True,True,True,True,True,True,True,False,False,False,False],
    "Martes": [True,True,True,True,True,True,True,True,True,True,True,False,False,False,False],
    "Miercoles": [True,True,True,True,True,True,True,True,True,True,True,False,False,False,False],
    "Jueves": [True,True,True,True,True,True,True,True,True,True,True,False,False,False,False],
    "Viernes": [True,True,True,True,True,True,True,True,True,True,True,False,False,False,False],
    "Sabado": [False,False,False,False,False,False,False,False,False,False,False,False,False,False,False]
})
disponibilidad = disponibilidadArr[4]

In [56]:
instancia1 = pg.GA(num_generations=20,
                   num_parents_mating=2,
                   fitness_func=obtenerDesempeno,
                   sol_per_pop=500,
                   num_genes=9,
                   parent_selection_type="sss",
                   keep_parents=1,
                   crossover_type='uniform',
                   mutation_type='random',
                   mutation_percent_genes=15,
                   gene_type=int,
                   gene_space=np.append(ofertaUtil.index.values,[-1]))
start_time = time.time()
instancia1.run()
tiempoejecucion = time.time() - start_time
print('Proceso terminado en ' + str(tiempoejecucion))
solution, solution_fitness, solution_idx = instancia1.best_solution()

Proceso terminado en 77.80124354362488


In [57]:
obtenerDatosCarga(solution,ofertaUtil)

,Clave,Nombre,Maestro,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
33,ID0204,Bases de Datos,Lara Peraza / Wilberth Eduardo,09:00-11:00,-,09:00-11:00,-,-,-
35,ID0206,Interconectividad en redes,Sansores Pérez / Candelaria Elizabeth,11:00-13:00,-,11:00-13:00,-,-,-
36,ID0263,Sistemas de planificación de recursos empresar...,Zavala López / Erika Erendira,13:00-14:00,-,13:00-15:00,-,-,-
39,IL0204,Estadística analítica,Cruz Pichardo / Felisa,07:00-09:00,-,07:00-09:00,-,-,-
40,DP0396,Identidad y cultura,Alavez San Pedro / Minerva,-,16:00-18:00,-,-,16:00-17:00,-
49,ID0205,Geometría computacional,Naredo García / Enrique,-,14:00-16:00,-,14:00-15:00,-,-
83,IT0317,Diseño de interfaces humano-máquina,Morales Saavedra / Emmanuel,-,-,15:00-16:00,-,14:00-16:00,-


In [58]:
evaluarSolucion(solution)

La carga evaluada tiene:
	22.63% de utilidad por las materias de primeros ciclos llevadas.
	0% del total de las materias reprobadas del estudiante.
	0% de costo por sobreposición con las horas donde el estudiante no está disponible.
	0.0% de costo por el total de horas libres por día.


In [59]:
obtenerHorario(solution,ofertaUtil)

,Hora,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
0,7:00-8:00,Estadística analítica,-,Estadística analítica,-,-,-
1,8:00-9:00,Estadística analítica,-,Estadística analítica,-,-,-
2,9:00-10:00,Bases de Datos,-,Bases de Datos,-,-,-
3,10:00-11:00,Bases de Datos,-,Bases de Datos,-,-,-
4,11:00-12:00,Interconectividad en redes,-,Interconectividad en redes,-,-,-
5,12:00-13:00,Interconectividad en redes,-,Interconectividad en redes,-,-,-
6,13:00-14:00,Sistemas de planificación de recursos empresar...,-,Sistemas de planificación de recursos empresar...,-,-,-
7,14:00-15:00,-,Geometría computacional,Sistemas de planificación de recursos empresar...,Geometría computacional,Diseño de interfaces humano-máquina,-
8,15:00-16:00,-,Geometría computacional,Diseño de interfaces humano-máquina,-,Diseño de interfaces humano-máquina,-
9,16:00-17:00,-,Identidad y cultura,-,-,Identidad y cultura,-


In [62]:
ofertaUtil[:10]

,Clave,Nombre,Maestro,Lunes,Martes,Miercoles,Jueves,Viernes,Sabado
32,DP0396,Identidad y cultura,Alavez San Pedro / Minerva,-,09:00-11:00,-,08:00-09:00,-,-
33,ID0204,Bases de Datos,Lara Peraza / Wilberth Eduardo,09:00-11:00,-,09:00-11:00,-,-,-
34,ID0205,Geometría computacional,Naredo García / Enrique,-,-,-,09:00-11:00,10:00-11:00,-
35,ID0206,Interconectividad en redes,Sansores Pérez / Candelaria Elizabeth,11:00-13:00,-,11:00-13:00,-,-,-
36,ID0263,Sistemas de planificación de recursos empresar...,Zavala López / Erika Erendira,13:00-14:00,-,13:00-15:00,-,-,-
37,ID0265,Programación web,Morales Saavedra / Emmanuel,13:00-14:00,-,13:00-15:00,-,-,-
38,II0320,Ecuaciones diferenciales,Mejenes Sánchez / Omar,-,07:00-09:00,-,07:00-08:00,-,-
39,IL0204,Estadística analítica,Cruz Pichardo / Felisa,07:00-09:00,-,07:00-09:00,-,-,-
40,DP0396,Identidad y cultura,Alavez San Pedro / Minerva,-,16:00-18:00,-,-,16:00-17:00,-
41,ID0204,Bases de Datos,Báez López / Yarely,18:00-20:00,-,17:00-19:00,-,-,-


In [64]:
evaluarSolucion([32,33,34,35,36,38,39])

La carga evaluada tiene:
	25.93% de utilidad por las materias de primeros ciclos llevadas.
	0% del total de las materias reprobadas del estudiante.
	0% de costo por sobreposición con las horas donde el estudiante no está disponible.
	0.0% de costo por el total de horas libres por día.
